In [77]:
import pandas as pd

In [79]:
df = pd.read_csv(r"C:\Users\hp\Downloads\Dataset_Uber Traffic.csv")

In [81]:
df.head()

,DateTime,Junction,Vehicles,ID
0,01/11/15 0:00,1,15,20151101001
1,01/11/15 1:00,1,13,20151101011
2,01/11/15 2:00,1,10,20151101021
3,01/11/15 3:00,1,7,20151101031
4,01/11/15 4:00,1,9,20151101041


In [83]:
# Data Cleaning
df.fillna(df.median(numeric_only=True), inplace=True)

In [85]:
df.drop_duplicates(inplace=True)

In [87]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

C:\Users\hp\AppData\Local\Temp\ipykernel_20524\2138640164.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['DateTime'])


In [88]:
if 'Junction' in df.columns:
    df['Junction'] = df['Junction'].astype('category')

In [89]:
df.set_index('DateTime', inplace=True)

In [93]:
hourly_df = df.groupby([pd.Grouper(freq='H'), 'Junction'], observed=True).sum().reset_index()

C:\Users\hp\AppData\Local\Temp\ipykernel_20524\1526364063.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = df.groupby([pd.Grouper(freq='H'), 'Junction'], observed=True).sum().reset_index()


In [95]:
hourly_df.head()

,DateTime,Junction,Vehicles,ID
0,2015-01-11 00:00:00,1,15,20151101001
1,2015-01-11 00:00:00,2,6,20151101002
2,2015-01-11 00:00:00,3,9,20151101003
3,2015-01-11 01:00:00,1,13,20151101011
4,2015-01-11 01:00:00,2,6,20151101012


In [97]:
# Normalizing the data
from sklearn.preprocessing import MinMaxScaler

In [99]:
scaler = MinMaxScaler()

In [101]:
hourly_df['Vehicle_Count_Scaled'] = scaler.fit_transform(hourly_df[['Vehicles']])

In [103]:
hourly_df[['DateTime', 'Junction', 'Vehicles', 'Vehicle_Count_Scaled']].head()

,DateTime,Junction,Vehicles,Vehicle_Count_Scaled
0,2015-01-11 00:00:00,1,15,0.078212
1,2015-01-11 00:00:00,2,6,0.027933
2,2015-01-11 00:00:00,3,9,0.044693
3,2015-01-11 01:00:00,1,13,0.067039
4,2015-01-11 01:00:00,2,6,0.027933


In [105]:
df.columns

Index(['Junction', 'Vehicles', 'ID'], dtype='object')

In [107]:
df['ID'].head()

DateTime
2015-01-11 00:00:00    20151101001
2015-01-11 01:00:00    20151101011
2015-01-11 02:00:00    20151101021
2015-01-11 03:00:00    20151101031
2015-01-11 04:00:00    20151101041
Name: ID, dtype: int64

In [109]:
df.rename(columns={'ID':'Time'}, inplace=True)

In [111]:
df['Time']= pd.to_datetime(df['Time'])

In [113]:
# Creating new features
df['Hour'] = df['Time'].dt.hour
df['DayOfWeek'] = df['Time'].dt.dayofweek  # 0 = Monday, 6 = Sunday
df['Month'] = df['Time'].dt.month

In [115]:
df['Is_Weekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

In [117]:
df.sort_values(['Junction', 'Time'], inplace=True)

In [119]:
df['Lag_1'] = df.groupby('Junction', observed=True)['Vehicles'].shift(1)
df['Lag_2'] = df.groupby('Junction', observed=True)['Vehicles'].shift(2)